# Neptune + LightGBM

## Before you start

### Install Dependencies

In [ ]:
pip install --quiet lightgbm==2.2.3 neptune-client neptune-contrib[monitoring]

### Import Libraries

In [ ]:
import lightgbm as lgb
import neptune
from neptunecontrib.monitoring.lightgbm import neptune_monitor

### Initialize Neptune

Neptune gives you an option of logging data under a public folder as an anonymous user. This is great when you are just trying out the application and don't have a Neptune account yet. 

In [ ]:
neptune.init(api_token='ANONYMOUS', project_qualified_name='shared/LightGBM-integration')

## Quickstart

**Note:** 


Instead of logging data to the public project 'shared/showroom' as an anonymous user 'neptuner' you can log it to your own project.

To do that:

1. Get your Neptune API token

  ![image](https://neptune.ai/wp-content/uploads/get_token.gif)

2. Pass the token to the `api_token` parameter of `neptune.init()` (learn how to do this securely [here](https://docs.neptune.ai/security-and-privacy/api-tokens/how-to-find-and-set-neptune-api-token.html))
3. Create a new Neptune Project (learn how to do that [here](https://docs.neptune.ai/workspace-project-and-user-management/projects/create-project.html))
4. Pass your username and project_name to the `project_qualified_name` parameter of `neptune.init()`

For example:

```
neptune.init(api_token='eyJhcGlfYW908fsdf23f940jiri0bn3085gh03riv03irn', project_qualified_name='siddhantsadangi/lightgbm')
```

### Step 1: Create an Experiment

This creates an experiment in Neptune.

Once you have a live experiment you can log things to it.


In [ ]:
neptune.create_experiment(name='LightGBM-training')

Click on the link above to open this experiment in Neptune.

For now it is empty but keep the tab with experiment open to see what happens next.

### Step 2: Pass ``neptune_monitor`` to ``lgb.train``

You can view the logging live in the Neptune tab once you run the below cell

In [ ]:
# Setting up a samplt lightGBM training job
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_wine

data = load_wine()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.1)
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

params = {'boosting_type': 'gbdt',
              'objective': 'multiclass',
              'num_class': 3,
              'num_leaves': 31,
              'learning_rate': 0.05,
              'feature_fraction': 0.9
              }

# Passing `neptune_monitor` to `lgb.train()`
gbm = lgb.train(params,
    lgb_train,
    num_boost_round=500,
    valid_sets=[lgb_train, lgb_eval],
    valid_names=['train','valid'],
    callbacks=[neptune_monitor()],
    )

### Step 3: Stop logging

When you track experiments with Neptune in Jupyter notebooks you need to explicitly stop the experiment by running ```neptune.stop()```.

If you are running Neptune in regular ```.py``` scripts it will stop automatically when your code stops running.

In [ ]:
neptune.stop()

## More Options

### Log Hyperparameters

You can log training and model hyperparameters. To do that just pass the parameter dictionary to the `create_experiment()` method

In [ ]:
params = {'boosting_type': 'gbdt',
              'objective': 'multiclass',
              'num_class': 3,
              'num_leaves': 31,
              'learning_rate': 0.05,
              'feature_fraction': 0.9
              }

# Log hyperparameters
neptune.create_experiment(name='LightGBM-training', params=params)

### Save Model Artifacts

You can log model weights to Neptune both during and after training.
To do that just use the `log_artifact()` method on the saved model file.

In [ ]:
gbm.save_model('lightgbm.pkl')

# Log model
neptune.log_artifact('lightgbm.pkl')

### Log Interactive Charts

You can log interactive charts to Neptune.

For example, let’s log an interactive ROC AUC curve.

#### 1. Install dependencies

In [ ]:
pip install --quiet neptune-contrib scikit-plot matplotlib==3.2.0 plotly==4.12.0

#### 2. Create an ROC AUC curve

In [ ]:
import matplotlib.pyplot as plt
from scikitplot.metrics import plot_roc

y_test_pred = gbm.predict(X_test)

fig, ax = plt.subplots()
plot_roc(y_test, y_test_pred, ax=ax)

#### 3. Log it to Neptune via `log_chart()` function.


In [ ]:
from neptunecontrib.api import log_chart

log_chart(name='ROC curve', chart=fig)

### Stop logging

In [ ]:
neptune.stop()

## Explore results in the Neptune UI